<a href="https://colab.research.google.com/github/saheedniyi02/fpl-ai/blob/master/clean_fixtures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
seasons=["2020-21","2021-22","2022-23"]
gameweek = 18
unused_columns=["xP","opponent_team","expected_assists","expected_goal_involvements","expected_goals",
                "expected_goals_conceded","element_type_ex","team_h_score","team_a_score","element" , "round" ,"fixture","starts" ]
#stats that you want to use their values for the last few games to predict a

no_last_stats=3 #number of last gameweek stats to use for prediction

history_stats=["assists","bonus","bps","clean_sheets","creativity","goals_conceded","goals_scored","ict_index","influence","minutes",
               "own_goals","penalties_missed","penalties_saved","red_cards","saves","selected","threat","total_points","transfers_in","transfers_out","value","match_result","team Goal scored","team Goal conceded"]

last_features=[f"last {no_last_stats} {stat}" for stat in history_stats]

mean_features=["assists","bonus","bps","clean_sheets","creativity","goals_conceded",
     "goals_scored","ict_index","influence","minutes"
     ,"own_goals","penalties_missed","penalties_saved",
     "red_cards","saves","selected","threat","total_points"
     ,"transfers_in","transfers_out","value","match_result","team Goal scored","team Goal conceded"]

std_features=["bps","creativity","ict_index","influence","minutes","selected","threat","total_points","transfers_in","transfers_out","value"]



dropped_columns = [
    "season",
    "opponent",
    "match_result",
    "position",
    "assists",
    "penalties_missed",
    "bonus",
    "bps",
    "clean_sheets",
    "creativity",
    "goals_conceded",
    "goals_scored",
    "ict_index",
    "influence",
    "own_goals",
    "penalties_saved",
    "red_cards",
    "saves",
    "selected",
    "threat",
    "transfers_balance",
    "transfers_in",
    "transfers_out",
    "yellow_cards"
]  # "value",



leak_columns = [
    "name",
    "team",
]  # columns that shouldnt be used in training fir fear of data leakage



short_term_stats=3

team_features=["team Goal scored","team Goal conceded"]

In [84]:
#from config import *
import numpy as np
import pandas as pd
from scipy import stats


def split_test(data, gameweek):
    # print(data["GW"].value_counts())
    data_gw = data[data["GW"] == gameweek]
    data_other_gw = data[~(data["GW"] == gameweek)]
    return data_gw, data_other_gw


def check_win(df):
    list_win = []
    for index in df.index:
        result = (df["team_a_score"] - df["team_h_score"]).loc[index]
        is_home = df["was_home"].loc[index]
        if result == 0:
            list_win.append(1)
        elif result > 0 and is_home == True:
            list_win.append(0)
        elif result < 0 and is_home == True:
            list_win.append(3)
        elif result > 0 and is_home == False:
            list_win.append(3)
        elif result < 0 and is_home == False:
            list_win.append(0)
        else:
            list_win.append(-1)
    return list_win


def get_2020_21_season_pos(club):
    """get the position of the club in the 2020-21 season"""
    if club == "Man City":
        return 1
    elif club == "Man Utd":
        return 2
    elif club == "Liverpool":
        return 3
    elif club == "Chelsea":
        return 4
    elif club == "Leicester":
        return 5
    elif club == "West Ham":
        return 6
    elif club == "Spurs":
        return 7
    elif club == "Arsenal":
        return 8
    elif club == "Leeds":
        return 9
    elif club == "Everton":
        return 10
    elif club == "Aston Villa":
        return 11
    elif club == "Newcastle":
        return 12
    elif club == "Wolves":
        return 13
    elif club == "Crystal Palace":
        return 14
    elif club == "Southampton":
        return 15
    elif club == "Brighton":
        return 16
    elif club == "Burnley":
        return 17
    else:
        return 20


def get_2019_20_season_pos(club):
    """get the position of the club in the 2019-20 season"""
    if club == "Liverpool":
        return 1
    elif club == "Man City":
        return 2
    elif club == "Man Utd":
        return 3
    elif club == "Chelsea":
        return 4
    elif club == "Leicester":
        return 5
    elif club == "Spurs":
        return 6
    elif club == "Wolves":
        return 7
    elif club == "Arsenal":
        return 8
    elif club == "Sheffield Utd":
        return 9
    elif club == "Burnley":
        return 10
    elif club == "Southampton":
        return 11
    elif club == "Everton":
        return 12
    elif club == "Newcastle":
        return 13
    elif club == "Crystal Palace":
        return 14
    elif club == "Brighton":
        return 15
    elif club == "West Ham":
        return 16
    elif club == "Aston Villa":
        return 17
    else:
        return 20


def get_2021_22_season_pos(club):
    if club == "Man City":
        return 1
    elif club == "Liverpool":
        return 2
    elif club == "Chelsea":
        return 3
    elif club == "Spurs":
        return 4
    elif club == "Arsenal":
        return 5
    elif club == "Man Utd":
        return 6
    elif club == "West Ham":
        return 7
    elif club == "Leicester":
        return 8
    elif club == "Brighton":
        return 9
    elif club == "Wolves":
        return 10
    elif club == "Newcastle":
        return 11
    elif club == "Crystal Palace":
        return 12
    elif club == "Brentford":
        return 13
    elif club == "Aston Villa":
        return 14
    elif club == "Southampton":
        return 15
    elif club == "Everton":
        return 16
    elif club == "Leeds":
        return 17
    else:
        return 20


def get_2022_23_season_pos(club):
    if club == "Man City":
        return 1
    elif club == "Arsenal":
        return 2
    elif club == "Man Utd":
        return 3
    elif club == "Newcastle":
        return 4
    elif club == "Liverpool":
        return 5
    elif club == "Brighton":
        return 6
    elif club == "Aston Villa":
        return 7
    elif club == "Spurs":
        return 8
    elif club == "Brentford":
        return 9
    elif club == "Fulham":
        return 10
    elif club == "Crystal Palace":
        return 11
    elif club == "Chelsea":
        return 12
    elif club == "Wolves":
        return 13
    elif club == "West Ham":
        return 14
    elif club == "Bournemouth":
        return 15
    elif club == "Nott'm Forest":
        return 16
    elif club == "Everton":
        return 17
    else:
        return 20


def get_last_season_pos(year):
    """get the function to get the last season position of a team at any year"""
    if year == "2020-21":
        return get_2019_20_season_pos
    elif year == "2021-22":
        return get_2020_21_season_pos
    elif year == "2022-23":
        return get_2021_22_season_pos
    elif year == "2023-24":
        return get_2022_23_season_pos


def remove_neg(val):
    if val > 0:
        return val
    else:
        return -val


def deque_and_queue(stats, value):
    # if -1 in stats:
    #  return stats
    # deque
    stats = stats[1:]
    stats.append(value)
    return stats


def get_last_stats(data, stat, name, no_last_stats=3):
    name_df = data[data["name"] == name]
    seasons = ["2020-21", "2021-22", "2022-23"]
    name_df_dict = {}
    for season in seasons:
        name_df_season = name_df[name_df["season"] == season]
        list_stats = []
        stats_x = []
        for value in name_df_season[stat]:
            if len(stats_x) < no_last_stats:
                list_stats.append(np.array(stats_x))
                stats_x.append(value)
            else:
                list_stats.append(np.array(stats_x))
                stats_x = deque_and_queue(stats_x, value)
        name_df_season[f"last {no_last_stats} {stat}"] = list_stats
        name_df_dict[season] = name_df_season
    return pd.concat(name_df_dict.values())


def get_all_players_last_stats(data, stat, no_last_stats=3):
    players_df = []
    for player in data["name"].unique():
        data_player = get_last_stats(data, stat, player, no_last_stats)
        players_df.append(data_player)
    # print(pd.concat(players_df))
    return pd.concat(players_df)


def get_last_stats_test(data, stat, name, no_last_stats=3):
    name_df = data[data["name"] == name]
    list_stats = []
    stats_x = []
    for value in name_df[stat]:
        if len(stats_x) < no_last_stats:
            list_stats.append(np.array(stats_x))
            stats_x.append(value)
        else:
            list_stats.append(np.array(stats_x))
            stats_x = deque_and_queue(stats_x, value)
    name_df[f"last {no_last_stats} {stat}"] = list_stats
    return name_df


def get_players_last_stats_test(data, stat, no_last_stats=short_term_stats):
    players_df = []
    for player in data["name"].unique():
        data_player = get_last_stats_test(data, stat, player, no_last_stats)
        players_df.append(data_player)
    # print(pd.concat(players_df))
    return pd.concat(players_df)

def get_all_stats_test(data, stat, name):
    name_df = data[data["name"] == name]
    list_stats = []
    stats_x = []
    for value in name_df[stat]:
        list_stats.append(np.array(stats_x))
        stats_x.append(value)
    name_df[f"all {stat}"] = list_stats
    return name_df


def get_players_all_stats_test(data, stat):
    players_df = []
    for player in data["name"].unique():
        data_player = get_all_stats_test(data, stat, player)
        players_df.append(data_player)
    # print(pd.concat(players_df))
    return pd.concat(players_df)

def convert_minutes(val):
    """CONVERTS MINUTES TO A CATEGORICAL OUTPUT"""
    if val > 10:
        return 1
    else:
        return 0


def find_mode(vals):
    """find the mode of vals"""
    try:
        if -1 in vals:
            return -1
        return stats.mode(vals)[0][0]
    except IndexError:
        return np.nan


def find_mean(vals):
    """find the mean of vals"""
    try:
        if -1 in vals:
            return -1
        return np.mean(vals)
    except:
        return np.nan


def find_max(vals):
    """find the maximum of vals"""
    try:
        if -1 in vals:
            return -1
        return np.max(vals)
    except:
        return np.nan


def find_std(vals):
    """find the standard deviation of vals"""
    try:
        if -1 in vals:
            return -1
        return np.std(vals)
    except:
        return np.nan


def find_value_count(vals, to_count):
    """find the number of times to_count appears in vals"""
    try:
        if -1 in vals:
            return -1
        values, count = np.unique(vals, return_counts=True)
        index = np.where(values == to_count)[0][0]
        return count[index]

    except:
        return -2


def get_opp_team(data):
    """get the opponent team for each player"""
    opp_teams = []
    home_teams = data["home_team"]
    away_teams = data["away_team"]
    my_teams = data["team"]
    for id in data.index:
        my_team = my_teams.iloc[id]
        home_team = home_teams.iloc[id]
        away_team = away_teams.iloc[id]
        if my_team == home_team:
            opp_teams.append(away_team)
        else:
            opp_teams.append(home_team)
    return opp_teams




def create_features(data, mean_features, std_features):
    for mean_feature in mean_features:
        print(mean_feature)
        data[f"mean {mean_feature} {no_last_stats}"] = [
            find_mean(values) for values in data[f"last {no_last_stats} {mean_feature}"]
        ]
        data[f"mean {mean_feature} all"] = [
            find_mean(values) for values in data[f"all {mean_feature}"]
        ]
        #data[f"mean {mean_feature} all"]= data[f"mean {mean_feature} all"]/data["GW"]
    for std_feature in std_features:
        print(std_feature)
        data[f"std {std_feature} {no_last_stats}"] = [
            find_std(values) for values in data[f"last {no_last_stats} {std_feature}"]
        ]
    return data


In [85]:


import pandas as pd
import numpy as np
#from config import unused_columns, history_stats, no_last_stats
import warnings

warnings.filterwarnings("ignore")


def calculate_ratio_team_value(name, df=None):
    """Calculate the ratio of player value to team value"""
    team = df[df["name"] == name]["team"].iloc[0]
    total_value = df[df["team"] == team]["value"].sum()
    value = df[df["name"] == name]["value"].iloc[0]
    return value * 100 / total_value


def calculate_position_rank(name, df=None):
    """Calculate the number of players with a higher  value"""
    value = df[df["name"] == name]["value"].iloc[0]
    position = df[df["name"] == name]["position"].iloc[0]
    team = df[df["name"] == name]["team"].iloc[0]
    return df[
        (df["value"] > value) & (df["position"] == position) & (df["team"] == team)
    ]["value"].shape[0]



old_gameweek_results = []
for i in range(1, gameweek):
    result = pd.read_csv(f"datasets/2023-24/results/GW{i}.csv", index_col=0)
    result.columns = [
        "element",
        "fixture",
        "opponent_team",
        "total_points",
        "was_home",
        "kickoff_time",
        "team_h_score",
        "team_a_score",
        "round",
        "minutes",
        "goals_scored",
        "assists",
        "clean_sheets",
        "goals_conceded",
        "own_goals",
        "penalties_saved",
        "penalties_missed",
        "yellow_cards",
        "red_cards",
        "saves",
        "bonus",
        "bps",
        "influence",
        "creativity",
        "threat",
        "ict_index",
        "starts",
        "expected_goals",
        "expected_assists",
        "expected_goal_involvements",
        "expected_goals_conceded",
        "value",
        "transfers_balance",
        "selected",
        "transfers_in",
        "transfers_out",
        "id",
        "name",
        "cost",
        "position",
        "home_team",
        "away_team",
        "team",
    ]
    year = "2023-24"
    previous_year = "2022-23"
    result["xP"] = 0
    result["GW"] = i
    result["match_result"] = check_win(result)
    result["opponent"] = get_opp_team(result)
    print(result.shape)
    result["last_season_position"] = result["team"].apply(get_last_season_pos(year))
    result["opponent_last_season_position"] = result["opponent"].apply(
        get_last_season_pos(year)
    )
    result["percent_value"] = result["name"].apply(
        calculate_ratio_team_value, args=(result,)
    )
    result["position rank"] = result["name"].apply(
        calculate_position_rank, args=(result,)
    )

    player_prev_stats = pd.read_csv(
        f"https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/{previous_year}/cleaned_players.csv"
    )
    player_prev_stats["name"] = (
        player_prev_stats["first_name"] + " " + player_prev_stats["second_name"]
    )
    player_prev_stats.drop(["first_name", "second_name"], axis=1, inplace=True)
    player_prev_stats.columns = player_prev_stats.columns + "_ex"
    result = pd.merge(
        result, player_prev_stats, left_on="name", right_on="name_ex", how="left"
    )
    result_home=result[result["was_home"]==True]
    result_away=result[result["was_home"]==False]
    result_home["team Goal scored"]=result_home["team_h_score"]
    result_home["team Goal conceded"]=result_home["team_a_score"]
    result_away["team Goal conceded"]=result_away["team_h_score"]
    result_away["team Goal scored"]=result_away["team_a_score"]
    result=pd.concat([result_home,result_away])
    result.drop(unused_columns, axis=1, inplace=True)
    result.fillna(0)
    old_gameweek_results.append(result)
    print(i)

# print(old_gameweek_results[0]["opponent"].value_counts())
current_fixture = pd.read_csv(
    f"datasets/2023-24/fixtures/GW{gameweek}.csv", index_col=0
)
current_fixture.columns = [
    "id",
    "name",
    "value",
    "position",
    "home_team",
    "away_team",
    "kickoff_time",
    "was_home",
    "team",
]


current_fixture["opponent"] = get_opp_team(current_fixture)
# print(current_fixture["opponent"].value_counts())
current_fixture["last_season_position"] = current_fixture["team"].apply(
    get_last_season_pos(year)
)
current_fixture["opponent_last_season_position"] = current_fixture["opponent"].apply(
    get_last_season_pos(year)
)
current_fixture["percent_value"] = current_fixture["name"].apply(
    calculate_ratio_team_value, args=(current_fixture,)
)
current_fixture["position rank"] = current_fixture["name"].apply(
    calculate_position_rank, args=(current_fixture,)
)


# check_win not posible because it's only fixtures we have for now
# merge last season data
player_prev_stats = pd.read_csv(
    f"https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/{previous_year}/cleaned_players.csv"
)

player_prev_stats["name"] = (
    player_prev_stats["first_name"] + " " + player_prev_stats["second_name"]
)

player_prev_stats.drop(["first_name", "second_name"], axis=1, inplace=True)
player_prev_stats.columns = player_prev_stats.columns + "_ex"
current_fixture = pd.merge(
    current_fixture, player_prev_stats, left_on="name", right_on="name_ex", how="left"
)

current_fixture["GW"] = gameweek
old_gameweek_results.append(current_fixture)
data = pd.concat(old_gameweek_results)

data = data.sort_values(["name", "kickoff_time"], ascending=True)


data["season"] = year
data.drop(["name_ex", "id", "home_team", "away_team"], axis=1, inplace=True)

# print(data[data["GW"]==17])

for stat in history_stats:
    print(stat)
    data = get_players_last_stats_test(data, stat)
    data = get_players_all_stats_test(data, stat)


data = create_features(data, mean_features, std_features)
data.drop(
    [f"last {no_last_stats} {stat}" for stat in history_stats], axis=1, inplace=True
)
data.drop([f"all {stat}" for stat in history_stats],axis=1,inplace=True)
data["ratio_goal_scored 3"]=((data["mean goals_scored 3"]/data["mean team Goal scored 3"]).replace(np.inf, 0))*100
data["ratio_goal_scored all"]=((data["mean goals_scored all"]/data["mean team Goal scored all"]).replace(np.inf, 0))*100


(613, 47)
1
(585, 47)
2
(405, 47)
3
(584, 47)
4
(585, 47)
5
(585, 47)
6
(658, 47)
7
(658, 47)
8
(658, 47)
9
(658, 47)
10
(722, 47)
11
(718, 47)
12
(717, 47)
13
(704, 47)
14
(686, 47)
15
(673, 47)
16
(658, 47)
17
     total_points  was_home          kickoff_time  minutes  goals_scored  \
132           0.0     False  2023-12-17T14:00:00Z      0.0           0.0   
612           0.0      True  2023-12-17T14:00:00Z      0.0           0.0   
109           0.0      True  2023-12-17T14:00:00Z      0.0           0.0   
15            0.0      True  2023-12-17T14:00:00Z      0.0           0.0   
449           0.0     False  2023-12-17T16:30:00Z      0.0           0.0   
..            ...       ...                   ...      ...           ...   
571           0.0     False  2023-12-15T20:00:00Z     69.0           0.0   
415           0.0      True  2023-12-16T15:00:00Z      0.0           0.0   
192           2.0      True  2023-12-16T17:30:00Z     90.0           0.0   
430           0.0     False 

In [86]:
# print(data[data["GW"]==17])
a=data[data["GW"]==17]
print(a['total_points'])

132    0.0
612    0.0
109    0.0
15     0.0
449    0.0
      ... 
571    0.0
415    0.0
192    2.0
430    0.0
231    0.0
Name: total_points, Length: 659, dtype: float64


In [87]:
#add opponent stats for gameweeks
list_data_gw=[]
for GW in range(1,gameweek + 1):

    data_gw=data[data["GW"]==GW].sort_values(by="mean team Goal scored all", ascending=False)
    data_opponent=data_gw.drop_duplicates(subset="team")[['team','mean team Goal scored 3','mean team Goal conceded 3','mean team Goal scored all','mean team Goal conceded all',"mean match_result all"]]
    # print(data_opponent.reset_index().drop("index",axis=1))
    #print(data_gw.shape)
    data_opponent.columns=['opponent','opp mean team Goal scored 3','opp mean team Goal conceded 3','opp mean team Goal scored all','opp mean team Goal conceded all',"opp mean match_result all"]
    #print(data_opponent.shape)
    data_gw=pd.merge(data_gw,data_opponent,on="opponent",how="left")
    list_data_gw.append(data_gw)

In [88]:
data=pd.concat(list_data_gw)
data=data.drop_duplicates(["GW","team","opponent","name","kickoff_time"])

for i in range(1, gameweek + 1):
    data_gw = data[data["GW"] == i]
    data_gw.to_csv(f"cleaned_dataset/2023-24/GW{i}.csv")
    print(i)

In [89]:
# data.shape
print(data[data["GW"]==17])

     total_points  was_home          kickoff_time  minutes  goals_scored  \
0             2.0      True  2023-12-16T15:00:00Z     90.0           0.0   
1             0.0      True  2023-12-16T15:00:00Z      0.0           0.0   
2             0.0      True  2023-12-16T15:00:00Z     90.0           0.0   
3             0.0      True  2023-12-16T15:00:00Z      0.0           0.0   
4             0.0      True  2023-12-16T15:00:00Z      0.0           0.0   
..            ...       ...                   ...      ...           ...   
654           0.0     False  2023-12-16T15:00:00Z      0.0           0.0   
655           2.0     False  2023-12-16T15:00:00Z     66.0           0.0   
656           1.0     False  2023-12-16T15:00:00Z     90.0           0.0   
657           1.0     False  2023-12-16T15:00:00Z     90.0           0.0   
658           1.0     False  2023-12-16T15:00:00Z     23.0           0.0   

     assists  clean_sheets  goals_conceded  own_goals  penalties_saved  ...  \
0       

In [90]:
# 5398+509

In [91]:
# data.shape

In [92]:
# save cleaned files
for i in range(1, gameweek + 1):
    data_gw = data[data["GW"] == i]
    data_gw.to_csv(f"cleaned_dataset/2023-24/GW{i}.csv")
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


# DEBUGGING ERRORS

In [46]:
# list(data.columns)
current_fixture

,id,name,value,position,home_team,away_team,kickoff_time,was_home,team,opponent,...,bps_ex,ict_index_ex,clean_sheets_ex,red_cards_ex,yellow_cards_ex,selected_by_percent_ex,now_cost_ex,element_type_ex,name_ex,GW
0,1,Folarin Balogun,44,FWD,Liverpool,Arsenal,2023-12-23T17:30:00Z,False,Arsenal,Liverpool,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17
1,2,Cédric Alves Soares,39,DEF,Liverpool,Arsenal,2023-12-23T17:30:00Z,False,Arsenal,Liverpool,...,56.0,11.9,0.0,0.0,0.0,0.1,42.0,DEF,Cédric Alves Soares,17
2,3,Mohamed Elneny,44,MID,Liverpool,Arsenal,2023-12-23T17:30:00Z,False,Arsenal,Liverpool,...,27.0,1.1,0.0,0.0,0.0,0.6,41.0,MID,Mohamed Elneny,17
3,4,Fábio Ferreira Vieira,54,MID,Liverpool,Arsenal,2023-12-23T17:30:00Z,False,Arsenal,Liverpool,...,134.0,41.4,2.0,0.0,0.0,0.1,56.0,MID,Fábio Ferreira Vieira,17
4,5,Gabriel dos Santos Magalhães,49,DEF,Liverpool,Arsenal,2023-12-23T17:30:00Z,False,Arsenal,Liverpool,...,723.0,126.5,14.0,0.0,5.0,13.4,52.0,DEF,Gabriel dos Santos Magalhães,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,704,Nathan Fraser,45,FWD,Wolves,Chelsea,2023-12-24T13:00:00Z,True,Wolves,Chelsea,...,0.0,0.0,0.0,0.0,0.0,0.1,45.0,FWD,Nathan Fraser,17
762,715,Jean-Ricner Bellegarde,49,MID,Wolves,Chelsea,2023-12-24T13:00:00Z,True,Wolves,Chelsea,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17
763,749,Matthew Whittingham,45,MID,Wolves,Chelsea,2023-12-24T13:00:00Z,True,Wolves,Chelsea,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17
764,750,Tawanda Chirewa,45,MID,Wolves,Chelsea,2023-12-24T13:00:00Z,True,Wolves,Chelsea,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17


In [33]:
for GW in range(1,gameweek + 1):
  print(f"GW{GW}")
  print(data[data["GW"]==GW]["team"].shape)

GW1
(613,)
GW2
(585,)
GW3
(405,)
GW4
(584,)
GW5
(585,)
GW6
(585,)
GW7
(658,)
GW8
(658,)
GW9
(658,)
GW10
(658,)
GW11
(722,)
GW12
(718,)
GW13
(717,)
GW14
(704,)
GW15
(686,)
GW16
(673,)
GW17
(765,)


In [34]:
for GW in range(1,gameweek + 1):
  print(f"GW{GW}")
  print(len(data[data["GW"]==GW]["team"].unique()))
  print(data[data["GW"]==GW].drop_duplicates(subset="team")[["team","opponent","last_season_position","mean team Goal scored all","opp mean team Goal scored all"]].reset_index().drop("index",axis=1))#.sort_values("last_season_position"))
  print("\n\n")
  #.sort_values(by="mean team Goal scored all", ascending=False)

GW1
20
              team        opponent  last_season_position  \
0            Spurs       Brentford                     8   
1         Brighton           Luton                     6   
2         West Ham     Bournemouth                    14   
3        Brentford           Spurs                     9   
4          Arsenal   Nott'm Forest                     2   
5          Man Utd          Wolves                     3   
6          Everton          Fulham                    17   
7          Chelsea       Liverpool                    12   
8    Sheffield Utd  Crystal Palace                    20   
9      Bournemouth        West Ham                    15   
10           Luton        Brighton                    20   
11       Liverpool         Chelsea                     5   
12          Fulham         Everton                    10   
13   Nott'm Forest         Arsenal                    16   
14       Newcastle     Aston Villa                     4   
15     Aston Villa       Newcastl

In [35]:
data[pd.isna(data["opp mean team Goal scored all"])]["GW"].value_counts()

1    582
3    174
2     72
5     44
4     34
6     34
Name: GW, dtype: int64

In [36]:
data[pd.isna(data["opp mean team Goal scored all"])]. shape

(940, 122)

In [37]:
data[pd.isna(data["opp mean team Goal scored all"])]["GW"].value_counts()

1    582
3    174
2     72
5     44
4     34
6     34
Name: GW, dtype: int64

In [38]:
data_gw[pd.isna(data_gw["opp mean team Goal scored all"])]["opponent"].value_counts()

Series([], Name: opponent, dtype: int64)

In [45]:
# data_gw[data_gw["opponent"]=="Crystal Palace"][["name","team"]]